In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error

In [2]:
#df_wages_male = pd.read_excel("employment/Wages_male.xls")
#df_wages_female = pd.read_excel("employment/Wages_female.xls")
#df_vulnerable_male = pd.read_excel("employment/Vulnerable_male.xls")
#df_vulnerable_female = pd.read_excel("employment/Vulnerable_female.xls")
#df_unemployment_15_male = pd.read_excel("employment/Unemployment_15_24_male.xls")
#df_unemployment_15_female = pd.read_excel("employment/Unemployment_15_24_female.xls")
#df_total_unemployment_male = pd.read_excel("employment/Total_Unemployment_male.xls")
#df_total_unemployment_female = pd.read_excel("employment/Total_Unemployment_female.xls")
#df_laborforce_male = pd.read_excel("employment/labor_force_participation_rate_male.xls")
#df_laborforce_female = pd.read_excel("employment/labor_force_participation_rate_female.xls")
df_services_male = pd.read_excel("employment/Employment_services_male.xls")
df_services_female = pd.read_excel("employment/Employment_services_female.xls")
df_agriculture_male = pd.read_excel("employment/Employment_agriculture_male.xls")
df_agriculture_female = pd.read_excel("employment/Employment_agriculture_female.xls")
df_industry_male = pd.read_excel("employment/Employment_indusrty_male.xls")
df_industry_female = pd.read_excel("employment/Employment_indusrty_female.xls")
#df_contributing_male = pd.read_excel("employment/Contributing_family _workers_male.xls")
#df_contributing_female = pd.read_excel("employment/Contributing_family _workers_female.xls")
df_economy_types = pd.read_excel("employment/economy_types.xlsx")
df_employment_female = pd.read_excel("employment/EmploymentToPopulationRatio_15+_female.xls")
df_employment_male = pd.read_excel("employment/EmploymentToPopulationRatio_15+_male.xls")

In [3]:
dict_female = {'Country Name' : df_economy_types['TableName']}
dict_male = {'Country Name' : df_economy_types['TableName']}
dict_GenderEmploymentIndex = {'Country Name' : df_economy_types['TableName'], 'Country Code' : df_economy_types['Country Code'],'IncomeGroup' : df_economy_types['IncomeGroup'], 'Region' : df_economy_types['Region'] }
dict_ServicesIndex = {'Country Name' : df_economy_types['TableName'], 'IncomeGroup' : df_economy_types['IncomeGroup'], 'Region' : df_economy_types['Region'] }
dict_IndustryIndex = {'Country Name' : df_economy_types['TableName'], 'IncomeGroup' : df_economy_types['IncomeGroup'], 'Region' : df_economy_types['Region'] }
dict_AgricultureIndex = {'Country Name' : df_economy_types['TableName'], 'IncomeGroup' : df_economy_types['IncomeGroup'], 'Region' : df_economy_types['Region'] }
j, k = 1, 0
for i in range(30):
    
    #Creating the string representation of the year
    if(i <=8):
        year_str = '199'+str(j)
        j +=1
    elif (i >= 9 and i <= 18) :
        year_str = '200'+str(k)
        k +=1
    else:
        year_str = '20'+str(k)
        k +=1
    
    #factor_wage = (df_wages_female[year_str]*df_employment_female[year_str])/(df_wages_male[year_str]*df_employment_male[year_str])
    #factor_laborforce = df_laborforce_female[year_str]/df_laborforce_male[year_str]
    factor_services = (df_services_female[year_str]*df_employment_female[year_str])/(df_services_male[year_str]*df_employment_male[year_str])
    factor_agriculture = (df_agriculture_female[year_str]*df_employment_female[year_str])/(df_agriculture_male[year_str]*df_employment_male[year_str])
    factor_industry = (df_industry_female[year_str]*df_employment_female[year_str])/(df_industry_male[year_str]*df_employment_male[year_str])
    factor_employment = df_employment_female[year_str]/df_employment_male[year_str]
    
    ratio =   factor_industry   +\
            factor_services   + factor_agriculture
    
    dict_GenderEmploymentIndex[year_str] = ratio/3
    dict_ServicesIndex[year_str] = factor_services
    dict_IndustryIndex[year_str] = factor_industry
    dict_AgricultureIndex[year_str] = factor_agriculture

#Creating the male, female and GEI dataframe 
df_female = pd.DataFrame(dict_female)
df_male = pd.DataFrame(dict_male)
df_GenderEmploymentIndex = pd.DataFrame(dict_GenderEmploymentIndex)
df_GenderServicesIndex = pd.DataFrame(dict_ServicesIndex)
df_GenderIndustryIndex = pd.DataFrame(dict_IndustryIndex)
df_GenderAgricultureIndex = pd.DataFrame(dict_AgricultureIndex)
print(df_GenderAgricultureIndex.head(5))
#df_GenderEmploymentIndex.to_csv (r'comboclustered_data.csv', index = None, header=True) 

           Country Name          IncomeGroup                      Region  \
0           Afghanistan           Low income                  South Asia   
1                Angola  Lower middle income          Sub-Saharan Africa   
2               Albania  Upper middle income       Europe & Central Asia   
3  United Arab Emirates          High income  Middle East & North Africa   
4             Argentina  Upper middle income   Latin America & Caribbean   

       1991      1992      1993      1994      1995      1996      1997  ...  \
0  0.231623  0.232199  0.231855  0.230641  0.233024  0.231250  0.227984  ...   
1  1.483473  1.476862  1.466940  1.471737  1.476816  1.483942  1.481946  ...   
2  1.051762  1.054694  1.037482  1.029736  1.030518  1.024728  1.043581  ...   
3  0.002195  0.002236  0.002284  0.002258  0.002240  0.002291  0.002309  ...   
4  0.175008  0.437392  0.325477  0.250945  0.275114  0.122606  0.137934  ...   

       2011      2012      2013      2014      2015      2016 

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

data = np.zeros(30)
i=0
for j in range(1991,2021):
    data[i] = df_GenderAgricultureIndex[str(j)][0]
    i = i+1
index= pd.date_range(start='1991', end='2021', freq='A')
agridata = pd.Series(data, index)
fit1 = ExponentialSmoothing(agridata,trend='add').fit()
fcast3 = fit1.forecast(5).rename(r'$\alpha=0.2$')
print(fcast3)


2021-12-31    0.506945
2022-12-31    0.515728
2023-12-31    0.524511
2024-12-31    0.533295
2025-12-31    0.542078
Freq: A-DEC, Name: $\alpha=0.2$, dtype: float64


In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
%matplotlib inline
rmseavg=0
MAPE=0
MAE=0
data_21= np.zeros(187)
data_22= np.zeros(187)
data_23= np.zeros(187)
data_24= np.zeros(187)
data_25= np.zeros(187)
for k in range(0,187):
    data = np.zeros(30)
    i=0
    for j in range(1991,2021):
        data[i] = df_GenderAgricultureIndex[str(j)][k]
        i = i+1
    index= pd.date_range(start='1991', end='2021', freq='A')
    agridata = pd.Series(data, index)
    fit3 = ExponentialSmoothing(agridata.iloc[0:25],trend='add').fit()
    fcast3 = fit3.forecast(5).rename(r'$\alpha=%s$'%fit3.model.params['smoothing_level'])
    mse1 = ((fcast3 - agridata.iloc[25:30]) ** 2).mean()
    rmseavg+= (round(np.sqrt(mse1), 2))
    Error = np.sum(np.abs(np.subtract(agridata.iloc[25:30],fcast3)))
    Average = np.sum(agridata.iloc[25:30])
    MAPE = MAPE+ (Error/Average)
    MAE = MAE+ mean_absolute_error(agridata.iloc[25:30], fcast3)
    fit4 = ExponentialSmoothing(agridata,trend='add').fit()
    fcast4 = fit4.forecast(5).rename(r'$\alpha=%s$'%fit3.model.params['smoothing_level'])
    #print(fcast4)
    data_21[k]= fcast4[0]
    data_22[k]= fcast4[1]
    data_23[k]= fcast4[2]
    data_24[k]= fcast4[3]
    data_25[k]= fcast4[4]
print('RMSE: '+ str(rmseavg/187))
print('MAPE: '+ str(MAPE/187))
print('MAE: '+ str(MAE/187))
df_GenderAgricultureIndex['2021']= data_21
df_GenderAgricultureIndex['2022']= data_22
df_GenderAgricultureIndex['2023']= data_23
df_GenderAgricultureIndex['2024']= data_24
df_GenderAgricultureIndex['2025']= data_25
print(df_GenderAgricultureIndex.head())

C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization faile

RMSE: 0.033529411764705815
MAPE: 0.08885516390175907
MAE: 0.0306324211712663
           Country Name          IncomeGroup                      Region  \
0           Afghanistan           Low income                  South Asia   
1                Angola  Lower middle income          Sub-Saharan Africa   
2               Albania  Upper middle income       Europe & Central Asia   
3  United Arab Emirates          High income  Middle East & North Africa   
4             Argentina  Upper middle income   Latin America & Caribbean   

       1991      1992      1993      1994      1995      1996      1997  ...  \
0  0.231623  0.232199  0.231855  0.230641  0.233024  0.231250  0.227984  ...   
1  1.483473  1.476862  1.466940  1.471737  1.476816  1.483942  1.481946  ...   
2  1.051762  1.054694  1.037482  1.029736  1.030518  1.024728  1.043581  ...   
3  0.002195  0.002236  0.002284  0.002258  0.002240  0.002291  0.002309  ...   
4  0.175008  0.437392  0.325477  0.250945  0.275114  0.122606  0.1

C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)


In [6]:
rmseavg=0
MAPE=0
MAE=0
data_21= np.zeros(187)
data_22= np.zeros(187)
data_23= np.zeros(187)
data_24= np.zeros(187)
data_25= np.zeros(187)
for k in range(0,187):
    data = np.zeros(30)
    i=0
    for j in range(1991,2021):
        data[i] = df_GenderIndustryIndex[str(j)][k]
        i = i+1
    index= pd.date_range(start='1991', end='2021', freq='A')
    industrydata = pd.Series(data, index)
    fit3 = ExponentialSmoothing(industrydata.iloc[0:25],trend='add').fit()
    fcast3 = fit3.forecast(5).rename(r'$\alpha=%s$'%fit3.model.params['smoothing_level'])
    mse1 = ((fcast3 - industrydata.iloc[25:30]) ** 2).mean()
    rmseavg+= (round(np.sqrt(mse1), 2))
    Error = np.sum(np.abs(np.subtract(industrydata.iloc[25:30],fcast3)))
    Average = np.sum(industrydata.iloc[25:30])
    MAPE = MAPE+ (Error/Average)
    MAE = MAE+ mean_absolute_error(industrydata.iloc[25:30], fcast3)
    fit4 = ExponentialSmoothing(industrydata,trend='add').fit()
    fcast4 = fit4.forecast(5).rename(r'$\alpha=%s$'%fit3.model.params['smoothing_level'])
    #print(fcast4)
    data_21[k]= fcast4[0]
    data_22[k]= fcast4[1]
    data_23[k]= fcast4[2]
    data_24[k]= fcast4[3]
    data_25[k]= fcast4[4]
print('RMSE: '+ str(rmseavg/187))
print('MAPE: '+ str(MAPE/187))
print('MAE: '+ str(MAE/187))
df_GenderIndustryIndex['2021']= data_21
df_GenderIndustryIndex['2022']= data_22
df_GenderIndustryIndex['2023']= data_23
df_GenderIndustryIndex['2024']= data_24
df_GenderIndustryIndex['2025']= data_25
print(df_GenderIndustryIndex.head())

C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization faile

RMSE: 0.023529411764705827
MAPE: 0.0683907090317672
MAE: 0.021734222968867196
           Country Name          IncomeGroup                      Region  \
0           Afghanistan           Low income                  South Asia   
1                Angola  Lower middle income          Sub-Saharan Africa   
2               Albania  Upper middle income       Europe & Central Asia   
3  United Arab Emirates          High income  Middle East & North Africa   
4             Argentina  Upper middle income   Latin America & Caribbean   

       1991      1992      1993      1994      1995      1996      1997  ...  \
0  0.236316  0.235417  0.234872  0.233596  0.225315  0.221824  0.217420  ...   
1  0.120134  0.119183  0.118281  0.116742  0.116433  0.115937  0.116437  ...   
2  0.349225  0.341447  0.330148  0.323784  0.312014  0.316821  0.310923  ...   
3  0.130004  0.130938  0.131903  0.130817  0.128280  0.130428  0.131853  ...   
4  0.292219  0.285097  0.255119  0.236742  0.257812  0.233427  0.

In [7]:
rmseavg=0
MAPE=0
MAE=0
data_21= np.zeros(187)
data_22= np.zeros(187)
data_23= np.zeros(187)
data_24= np.zeros(187)
data_25= np.zeros(187)
for k in range(0,187):
    data = np.zeros(30)
    i=0
    for j in range(1991,2021):
        data[i] = df_GenderServicesIndex[str(j)][k]
        i = i+1
    index= pd.date_range(start='1991', end='2021', freq='A')
    servicesdata = pd.Series(data, index)
    fit3 = ExponentialSmoothing(servicesdata.iloc[0:25],trend='add').fit()
    fcast3 = fit3.forecast(5).rename(r'$\alpha=%s$'%fit3.model.params['smoothing_level'])
    mse1 = ((fcast3 - servicesdata.iloc[25:30]) ** 2).mean()
    rmseavg+= (round(np.sqrt(mse1), 2))
    Error = np.sum(np.abs(np.subtract(servicesdata.iloc[25:30],fcast3)))
    Average = np.sum(servicesdata.iloc[25:30])
    MAPE = MAPE+ (Error/Average)
    MAE = MAE+ mean_absolute_error(servicesdata.iloc[25:30], fcast3)
    fit4 = ExponentialSmoothing(servicesdata,trend='add').fit()
    fcast4 = fit4.forecast(5).rename(r'$\alpha=%s$'%fit3.model.params['smoothing_level'])
    #print(fcast4)
    data_21[k]= fcast4[0]
    data_22[k]= fcast4[1]
    data_23[k]= fcast4[2]
    data_24[k]= fcast4[3]
    data_25[k]= fcast4[4]
print('RMSE: '+ str(rmseavg/187))
print('MAPE: '+ str(MAPE/187))
print('MAE: '+ str(MAE/187))
df_GenderServicesIndex['2021']= data_21
df_GenderServicesIndex['2022']= data_22
df_GenderServicesIndex['2023']= data_23
df_GenderServicesIndex['2024']= data_24
df_GenderServicesIndex['2025']= data_25
print(df_GenderServicesIndex.head())

C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
C:\Users\Srihaasa\Anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:711: ConvergenceWarning: Optimization faile

RMSE: 0.029037433155080138
MAPE: 0.03307640355354005
MAE: 0.026221340753484035
           Country Name          IncomeGroup                      Region  \
0           Afghanistan           Low income                  South Asia   
1                Angola  Lower middle income          Sub-Saharan Africa   
2               Albania  Upper middle income       Europe & Central Asia   
3  United Arab Emirates          High income  Middle East & North Africa   
4             Argentina  Upper middle income   Latin America & Caribbean   

       1991      1992      1993      1994      1995      1996      1997  ...  \
0  0.058587  0.059405  0.060032  0.060638  0.061228  0.060934  0.060209  ...   
1  0.910595  0.907094  0.901224  0.893557  0.901133  0.907514  0.921324  ...   
2  0.593987  0.597561  0.593492  0.593984  0.581557  0.560335  0.566217  ...   
3  0.493786  0.507388  0.522924  0.526718  0.531010  0.541130  0.552347  ...   
4  0.831269  0.850851  0.825826  0.852266  0.783872  0.829182  0

In [8]:
data_21= np.zeros(187)
data_22= np.zeros(187)
data_23= np.zeros(187)
data_24= np.zeros(187)
data_25= np.zeros(187)
for k in range(0,187):
    data_21[k]= (df_GenderAgricultureIndex['2021'][k]+ df_GenderIndustryIndex['2021'][k]+ df_GenderServicesIndex['2021'][k])/3
    data_22[k]= (df_GenderAgricultureIndex['2022'][k]+ df_GenderIndustryIndex['2022'][k]+ df_GenderServicesIndex['2022'][k])/3
    data_23[k]= (df_GenderAgricultureIndex['2023'][k]+ df_GenderIndustryIndex['2023'][k]+ df_GenderServicesIndex['2023'][k])/3
    data_24[k]= (df_GenderAgricultureIndex['2024'][k]+ df_GenderIndustryIndex['2024'][k]+ df_GenderServicesIndex['2024'][k])/3
    data_25[k]= (df_GenderAgricultureIndex['2025'][k]+ df_GenderIndustryIndex['2025'][k]+ df_GenderServicesIndex['2025'][k])/3
df_GenderEmploymentIndex['2021']= data_21
df_GenderEmploymentIndex['2022']= data_22
df_GenderEmploymentIndex['2023']= data_23
df_GenderEmploymentIndex['2024']= data_24
df_GenderEmploymentIndex['2025']= data_25
print(df_GenderEmploymentIndex.head())    

           Country Name Country Code          IncomeGroup  \
0           Afghanistan          AFG           Low income   
1                Angola          AGO  Lower middle income   
2               Albania          ALB  Upper middle income   
3  United Arab Emirates          ARE          High income   
4             Argentina          ARG  Upper middle income   

                       Region      1991      1992      1993      1994  \
0                  South Asia  0.175509  0.175674  0.175586  0.174958   
1          Sub-Saharan Africa  0.838068  0.834380  0.828815  0.827345   
2       Europe & Central Asia  0.664991  0.664567  0.653707  0.649168   
3  Middle East & North Africa  0.208662  0.213521  0.219037  0.219931   
4   Latin America & Caribbean  0.432832  0.524447  0.468807  0.446651   

       1995      1996  ...      2016      2017      2018      2019      2020  \
0  0.173189  0.171336  ...  0.278332  0.306192  0.316516  0.321914  0.326837   
1  0.831461  0.835798  ...  0.7671

In [9]:
df_GenderEmploymentIndex['pre5yearaverage'] = df_GenderEmploymentIndex[['2016','2017','2018','2019','2020']].mean(axis=1)
df_GenderEmploymentIndex['post5yearaverage'] = df_GenderEmploymentIndex[['2021','2022','2023','2024','2025']].mean(axis=1)


In [12]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Choropleth(
    locations = df_GenderEmploymentIndex['Country Code'],
    z = df_GenderEmploymentIndex['post5yearaverage'],
    text = df_GenderEmploymentIndex['Country Name'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '',
    colorbar_title = 'GEPI',
))

fig.update_layout(
    title_text='Average forecasted GEPI from 2021 to 2025- Double Exponential Smoothing',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
    
)

fig.show()

In [11]:
df_GenderEmploymentIndex.to_csv (r'comboclustered_data2.csv', index = None, header=True) 